In [96]:
import os
import pandas as pd
path_to_incubations = "../../lab_work/sessions/incubations"


def load_incubation_data(path_to_incubations):
    dfs = []
    for file_name in os.listdir(path_to_incubations):
        path_to_file = os.path.join(path_to_incubations, file_name)
        print(path_to_file)
        dfs.append(pd.read_csv(path_to_file))

    result = pd.concat(dfs, ignore_index=True)

    return result
    # print(path_to_file)
    
def tidy_incubation_data(df):
    df['sample_name'] = df['sample_id']
    df['sample_id'] = df.apply(lambda row: set_sample_id(row), axis=1)
    df = df.drop(['sample_name'], axis=1)
    return df
    



def set_sample_id(row):
    sample_name = row['sample_name']
    if sample_name.startswith('40ml'):
        return f"40mL_{sample_name[-3:]}"
    else:
        return f"2mL_{sample_name}"
        
incubation_df = load_incubation_data(path_to_incubations=path_to_incubations)
incubation_df = tidy_incubation_data(incubation_df)
incubation_df

../../lab_work/sessions/incubations/2mLWetSpirullina.csv
../../lab_work/sessions/incubations/40mLDrySpirullina.csv
../../lab_work/sessions/incubations/2mLDrySpirullina.csv
../../lab_work/sessions/incubations/40mLWetSpirullina.csv


,sample_id,ratio,intended weight (mg),weight (mg),notes,incubation_start_date,biomass (mg),salt (mg),water (mg),innoculum added
0,2mL_2.1.0,01:00,109,129.0,NaN,7/12/23,NaN,NaN,NaN,NaN
1,2mL_2.1.1,01:00,109,119.0,NaN,7/12/23,NaN,NaN,NaN,NaN
2,2mL_2.1.2,01:00,109,165.0,NaN,7/12/23,NaN,NaN,NaN,NaN
3,2mL_2.2.0,01:01,215,350.0,NaN,7/12/23,NaN,NaN,NaN,NaN
4,2mL_2.2.1,01:01,215,248.0,NaN,7/12/23,NaN,NaN,NaN,NaN
5,2mL_2.2.2,01:01,215,241.0,NaN,7/12/23,NaN,NaN,NaN,NaN
6,2mL_2.3.0,16:01,115,118.0,NaN,7/12/23,NaN,NaN,NaN,NaN
7,2mL_2.3.1,16:01,115,121.0,NaN,7/12/23,NaN,NaN,NaN,NaN
8,2mL_2.3.2,16:01,115,124.0,NaN,7/12/23,NaN,NaN,NaN,NaN
9,2mL_2.4.0,01:05,635,707.0,NaN,7/12/23,NaN,NaN,NaN,NaN


In [4]:
import numpy as np
data_df = pd.read_csv('../../GCTCD_PDF_READER/GCPDFReader/csvs/tidied_1692821404.csv')

# np.unique(data_df['sample_id'])
data_df

,Unnamed: 0.1,Unnamed: 0,Sample_Name,Sample_Date,Instrument,Peak,Time,Type,Area,Height,Width,Start,End,pdf_file_name,sample_id,peak_compound,is_std,known_conc,calculated_conc
0,0,0,40ML1_10,8/16/2023,GCTCD,1,3.151,BP,494087.00000,188091.00000,0.0386,3.107,3.267,../../lab_work/sessions/GCTCD/20230816/40mL_1....,40mL_1.1,NaN,False,NaN,NaN
1,1,1,40ML1_10,8/16/2023,GCTCD,2,4.693,PP,1595.22729,652.50348,0.0375,4.642,4.791,../../lab_work/sessions/GCTCD/20230816/40mL_1....,40mL_1.1,CO2,False,NaN,934.370010
2,2,2,40ML1_10,8/16/2023,GCTCD,3,5.270,BP,124.49847,48.04475,0.0332,5.238,5.311,../../lab_work/sessions/GCTCD/20230816/40mL_1....,40mL_1.1,NaN,False,NaN,NaN
3,3,3,40ML1_10,8/16/2023,GCTCD,4,5.853,PV,188.87746,73.72960,0.0339,5.817,5.888,../../lab_work/sessions/GCTCD/20230816/40mL_1....,40mL_1.1,NaN,False,NaN,NaN
4,4,4,40ML1_10,8/16/2023,GCTCD,5,5.961,VV,529.92957,109.93177,0.0602,5.888,6.011,../../lab_work/sessions/GCTCD/20230816/40mL_1....,40mL_1.1,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,424,35,2.979pp m,8/14/2023,GCFID,1,2.387,MF,240.27603,31.48297,0.1272,0.000,2.495,../../lab_work/sessions/GCFID/20230814/2023081...,CO2_600ppm_CH4_2179ppb,CO,True,2.031,NaN
425,425,36,2.979pp m,8/14/2023,GCFID,2,2.495,FM,7.25621,4.14673,0.0292,2.495,2.521,../../lab_work/sessions/GCFID/20230814/2023081...,CO2_600ppm_CH4_2179ppb,NaN,True,NaN,NaN
426,426,37,2.979pp m,8/14/2023,GCFID,3,2.567,FM,185.20935,15.03130,0.2054,2.521,4.998,../../lab_work/sessions/GCFID/20230814/2023081...,CO2_600ppm_CH4_2179ppb,CH4,True,2.979,178.510783
427,427,38,2.979pp m,8/14/2023,GCFID,1,2.385,BV,120.97746,26.07795,0.0561,2.310,2.493,../../lab_work/sessions/GCFID/20230814/2023081...,CO2_600ppm_CH4_2179ppb,CO,True,2.031,NaN


In [35]:
new_df = pd.merge(data_df, incubation_df, on='sample_id', how='left')
# new_df[new_df['Sample_Name']=='2ML_2_10']['sample_id']
new_df['ratio']

pandas.core.series.Series

In [5]:
np.unique(new_df[new_df['incubation_start_date'].isnull()]['sample_id'])

array(['AMBIENT_AIR', 'CH4_2pph', 'CO2_20pph', 'CO2_600ppm_CH4_2179ppb',
       'DROP_ME'], dtype=object)

In [95]:

from datetime import datetime

def set_incubation_length(row):
    try:
        sample_date  = datetime.strptime(row['Sample_Date'], '%m/%d/%Y')
        incubation_start_date  = datetime.strptime(row['incubation_start_date'], '%m/%d/%y')
        return (sample_date - incubation_start_date).days

    except Exception as e:
        # print(e, row['Sample_Date'], row['incubation_start_date'])
        return None
    
def set_salt_to_biomass(row):
    if row['is_std']:
        return None
    if row['sample_id'] == 'DROP_ME':
        return None
    if row['ratio'] == "01:00":
        return 0
    if row['ratio'] == "01:01":
        return 1
    if row['ratio'] == "16:01":
        return 1.0/16
    if row['ratio'] == "01:05":
        return 5
    
def take_log_of_salt_ratio(row):
    print(row['salt_ratio'])
    if row['salt_ratio'] == None:
        return None
    else:
        x = np.log10(row['salt_ratio'])
        print(x)
        return x

new_df['incubation_length'] = new_df.apply(lambda row: set_incubation_length(row), axis=1)
new_df['salt_ratio'] = new_df.apply(lambda row: set_salt_to_biomass(row), axis=1)
new_df['salt_ratio_log'] = new_df.apply(lambda row: take_log_of_salt_ratio(row), axis=1)

new_df

0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
0.0
-inf
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
0.0625
-1.2041199826559248
0.0625
-1.2041199826559248
0.0625
-1.2041199826559248
0.0625
-1.2041199826559248
5.0
0.6989700043360189
5.0
0.6989700043360189
5.0
0.6989700043360189
5.0
0.6989700043360189
0.0
-inf
0.0
-inf
1.0
0.0
1.0
0.0
1.0
0.0
0.0625
-1.2041199826559248
0.0625
-1.2041199826559248
5.0
0.6989700043360189
5.0
0.6989700043360189
5.0
0.6989700043360189
5.0
0.6989700043360189
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
n

/var/folders/f3/zyvmf_xs32qbkl_h7913df540000gn/T/ipykernel_7493/2356663063.py:32: RuntimeWarning: divide by zero encountered in log10
  x = np.log10(row['salt_ratio'])


,Unnamed: 0.1,Unnamed: 0,Sample_Name,Sample_Date,Instrument,Peak,Time,Type,Area,Height,...,weight (mg),notes,incubation_start_date,biomass (mg),salt (mg),water (mg),innoculum added,incubation_length,salt_ratio,salt_ratio_log
0,0,0,40ML1_10,8/16/2023,GCTCD,1,3.151,BP,494087.00000,188091.00000,...,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51.0,0.0,-inf
1,1,1,40ML1_10,8/16/2023,GCTCD,2,4.693,PP,1595.22729,652.50348,...,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51.0,0.0,-inf
2,2,2,40ML1_10,8/16/2023,GCTCD,3,5.270,BP,124.49847,48.04475,...,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51.0,0.0,-inf
3,3,3,40ML1_10,8/16/2023,GCTCD,4,5.853,PV,188.87746,73.72960,...,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51.0,0.0,-inf
4,4,4,40ML1_10,8/16/2023,GCTCD,5,5.961,VV,529.92957,109.93177,...,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51.0,0.0,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,424,35,2.979pp m,8/14/2023,GCFID,1,2.387,MF,240.27603,31.48297,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,425,36,2.979pp m,8/14/2023,GCFID,2,2.495,FM,7.25621,4.14673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,426,37,2.979pp m,8/14/2023,GCFID,3,2.567,FM,185.20935,15.03130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,427,38,2.979pp m,8/14/2023,GCFID,1,2.385,BV,120.97746,26.07795,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
relevant_df  = new_df[(new_df['peak_compound'].notnull()) & (new_df['sample_id']!="DROP_ME")]
methane_df = relevant_df[(relevant_df['peak_compound'] == 'CH4') & (relevant_df['is_std'] == False)]
CO2_df = relevant_df[(relevant_df['peak_compound'] == 'CO2') & (relevant_df['is_std'] == False)]
CO2_df['salt_ratio']


1      0.0000
24     1.0000
31     0.0625
33     5.0000
37     0.0000
39     1.0000
42     0.0625
44     5.0000
100    0.0000
104    0.0000
106    0.0000
108    1.0000
110    1.0000
116    1.0000
118    0.0625
123    0.0625
127    0.0625
133    5.0000
137    5.0000
142    5.0000
176    0.0000
179    0.0000
181    0.0000
186    1.0000
189    1.0000
191    1.0000
195    0.0625
197    0.0625
200    0.0625
204    5.0000
208    5.0000
210    5.0000
307    0.0000
311    1.0000
313    0.0625
316    5.0000
324    0.0000
326    1.0000
328    0.0625
Name: salt_ratio, dtype: float64

In [52]:
methane_df

,Unnamed: 0.1,Unnamed: 0,Sample_Name,Sample_Date,Instrument,Peak,Time,Type,Area,Height,...,intended weight (mg),weight (mg),notes,incubation_start_date,biomass (mg),salt (mg),water (mg),innoculum added,incubation_length,salt_ratio
400,400,11,40ML2_00,8/16/2023,GCFID,2,2.565,VP,709.21515,238.68648,...,1750.0,NaN,NaN,7/12/23,1750.0,0.0,2975.0,~100 mg sand from death valley,35.0,0.0000
402,402,13,40ML2_01,8/16/2023,GCFID,2,2.567,VB,662.82294,233.42636,...,1750.0,NaN,NaN,7/12/23,1750.0,0.0,2975.0,~100 mg sand from death valley,35.0,0.0000
404,404,15,40ML2_10,8/16/2023,GCFID,2,2.569,VB,287.63409,70.53904,...,3500.0,NaN,NaN,7/12/23,1750.0,1750.0,5950.0,~100 mg sand from death valley,35.0,1.0000
406,406,17,40ML2_11,8/16/2023,GCFID,2,2.567,VP,270.27475,63.39441,...,3500.0,NaN,NaN,7/12/23,1750.0,1750.0,5950.0,~100 mg sand from death valley,35.0,1.0000
408,408,19,40ML2_20,8/16/2023,GCFID,2,2.567,VP,423.02002,126.01544,...,1860.0,NaN,NaN,7/12/23,1750.0,110.0,3162.0,~100 mg sand from death valley,35.0,0.0625
410,410,21,40ML2_21,8/16/2023,GCFID,2,2.568,VB,423.75296,131.45924,...,1860.0,NaN,NaN,7/12/23,1750.0,110.0,3162.0,~100 mg sand from death valley,35.0,0.0625


In [89]:
np.unique(CO2_df['salt_ratio_log'])


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
      dtype=object)

In [87]:
from bokeh.models import Range1d
import pandas as pd
import numpy as np
import bokeh.io
from bokeh.models import  ColumnDataSource,Range1d, LabelSet, Label

from bokeh.io import export_png
from bokeh import plotting
from bokeh.models import LogColorMapper
from bokeh.models import ColorBar
bokeh.io.output_notebook()




def create_chart(title, df):
    df = ColumnDataSource(df)
    exp_cmap = LogColorMapper(palette="Viridis256")
    TOOLTIPS = [
    ("sample_id", "@sample_id"),
    ("salt_ratio", "@salt_ratio")
        ]
    p = plotting.figure(
    # frame_height=500,
    # frame_width=800,
    x_axis_label = "Incubation Length",
    y_axis_label = "Calculated Conc (ppm)",
    # y_axis_type="log",
    title=title,
    toolbar_location='above',
    tooltips = TOOLTIPS
    )
    p.add_layout(bokeh.models.Legend(), "right")
    # p.legend.click_policy = "hide"

    p.circle(
            source=df,
            x = "incubation_length",
            y = "calculated_conc",
            # color=color,
            line_width = 5,
            color={"field":"salt_ratio_log", "transform":exp_cmap},
        )
    bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
    p.add_layout(bar, "right")
    return p

hold = create_chart("CO2 conc", CO2_df)
bokeh.io.show(hold)


Loading BokehJS ...

In [73]:
hold = create_chart("Methane conc", methane_df)
bokeh.io.show(hold)